<a href="https://colab.research.google.com/github/ZeroLeon/NLP_Project_with_FastAI/blob/master/text_sum_with_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os 
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/My Drive/fastai'
path = Path(base_dir +'/data/txt_sum_ex')
path.mkdir(parents=True,exist_ok=True)
os.chdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [0]:
df = pd.read_csv('datascience.csv', encoding='gb18030')

In [0]:
!pip install jieba
!pip install pyldavis

     |████████████████████████████████| 1.6MB 2.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


In [0]:
import jieba

In [0]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [0]:
df["content_cutted"] = df.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.121 seconds.
Prefix dict has been built succesfully.


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [0]:
n_features = 1000

In [0]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [0]:
from sklearn.decomposition import LatentDirichletAllocation

In [0]:
n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [0]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=50,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [0]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [0]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
学习 模型 使用 算法 方法 机器 可视化 神经网络 特征 处理 不同 计算 用户 数据库 系统 如果 分类 训练 一种 基于
Topic #1:
这个 就是 可能 没有 如果 他们 自己 很多 什么 不是 但是 或者 因为 时候 这样 现在 电子 一些 所以 孩子
Topic #2:
企业 平台 服务 管理 互联网 数据分析 公司 产品 用户 业务 行业 客户 金融 创新 实现 价值 系统 能力 工作 需求
Topic #3:
中国 2016 市场 增长 10 城市 用户 2015 关注 行业 其中 30 人口 检索 阅读 大众 投资 全国 美国 20
Topic #4:
人工智能 学习 领域 智能 机器人 机器 人类 公司 深度 研究 未来 识别 已经 系统 计算机 目前 医疗 语音 方面 服务



In [0]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.149635  0.021451       1        1  25.852071
2     -0.100942  0.102186       2        1  25.798362
1      0.127037 -0.112524       3        1  21.632325
4     -0.005510  0.118440       4        1  14.625269
3     -0.170219 -0.129553       5        1  12.091972, topic_info=    Category         Freq  Term        Total  loglift  logprob
122  Default  2771.000000  人工智能  2771.000000  30.0000  30.0000
364  Default  4194.000000    学习  4194.000000  29.0000  29.0000
148  Default  3220.000000    企业  3220.000000  28.0000  28.0000
615  Default  2178.000000    机器  2178.000000  27.0000  27.0000
586  Default  1450.000000    智能  1450.000000  26.0000  26.0000
616  Default  1109.000000   机器人  1109.000000  25.0000  25.0000
91   Default  1773.000000    中国  1773.000000  24.0000  24.0000
992  Default  2003.000000    领域  2003.000000  23.0000  23.0000
125  Default  1017.000000    人类  1017.000000  22.0000  22.0000
929  Default  2265.000000    这个  2265.000000  21.0000  21.0000
645  Default  1765.000000    模型  1765.000000  20.0000  20.0000
778  Default  1839.000000    算法  1839.000000  19.0000  19.0000
18   Default  1004.000000  2016  1004.000000  18.0000  18.0000
193  Default  2440.000000    公司  2440.000000  17.0000  17.0000
779  Default  1362.000000    管理  1362.000000  16.0000  16.0000
401  Default  1798.000000    就是  1798.000000  15.0000  15.0000
678  Default  1336.000000    深度  1336.000000  14.0000  14.0000
417  Default  1411.000000    市场  1411.000000  13.0000  13.0000
340  Default   814.000000    增长   814.000000  12.0000  12.0000
607  Default  1748.000000    服务  1748.000000  11.0000  11.0000
331  Default   798.000000    城市   798.000000  10.0000  10.0000
424  Default  1734.000000    平台  1734.000000   9.0000   9.0000
129  Default  1067.000000    什么  1067.000000   8.0000   8.0000
1    Default   979.000000    10   979.000000   7.0000   7.0000
85   Default  1121.000000    业务  1121.000000   6.0000   6.0000
360  Default  1663.000000    如果  1663.000000   5.0000   5.0000
454  Default  1239.000000    很多  1239.000000   4.0000   4.0000
845  Default  1589.000000    行业  1589.000000   3.0000   3.0000
368  Default   626.000000    孩子   626.000000   2.0000   2.0000
718  Default   727.000000    电子   727.000000   1.0000   1.0000
..       ...          ...   ...          ...      ...      ...
187   Topic5   302.316730    全国   390.684292   1.8562  -5.1199
609   Topic5   136.545315    期间   177.943139   1.8478  -5.9147
8     Topic5   242.205138    15   317.223026   1.8428  -5.3416
716   Topic5   198.114045    申请   259.964435   1.8409  -5.5425
21    Topic5   163.196166    25   215.494631   1.8346  -5.7364
326   Topic5   236.632356    地区   313.108847   1.8326  -5.3648
340   Topic5   610.327748    增长   814.615688   1.8239  -4.4174
18    Topic5   750.205913  2016  1004.067948   1.8212  -4.2110
22    Topic5   326.256365    30   448.508389   1.7944  -5.0437
17    Topic5   485.246729  2015   708.467452   1.7342  -4.6467
331   Topic5   534.501579    城市   798.602410   1.7111  -4.5500
91    Topic5  1047.305694    中国  1773.569103   1.5859  -3.8774
1     Topic5   591.436906    10   979.981143   1.6077  -4.4488
4     Topic5   255.351191    11   352.936029   1.7890  -5.2887
417   Topic5   640.828649    市场  1411.161873   1.3232  -4.3686
250   Topic5   248.074821    北京   366.565127   1.7222  -5.3176
969   Topic5   311.601217    阅读   562.099252   1.5227  -5.0896
228   Topic5   238.056304    分别   349.847809   1.7276  -5.3588
13    Topic5   296.282344    20   546.958099   1.4996  -5.1400
198   Topic5   382.407822    关注   931.671734   1.2221  -4.8849
906   Topic5   272.897849    超过   500.115946   1.5069  -5.2222
713   Topic5   523.213916    用户  2486.159933   0.5541  -4.5714
203   Topic5   330.011311    其中   825.447658   1.1958  -5.0322
811   Topic5   299.533231    美国   651.192038   1.3360  -5.1291
498   Topic5   307.674314    投资   773.185668   1.19